In [4]:
from google.colab import files
uploaded = files.upload()

Saving chatgpt_generated_data_with_class.csv to chatgpt_generated_data_with_class.csv


In [17]:
import io
import pandas as pd
#data = pd.read_csv(io.BytesIO(uploaded['FIRE2023_IRSE_training_Code_Comment_Seed_Data (1).csv']))
data = pd.read_csv('FIRE2023_IRSE_training_Code_Comment_Seed_Data.csv')
data.head(5)

,Comments,Surrounding Code Context,Class
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful


In [18]:
data.describe()

,Comments,Surrounding Code Context,Class
count,11452,11452,11452
unique,7107,7791,2
top,/*********************************************...,-10. // the X-windows Version 11 distribution...,Useful
freq,144,72,7063


In [19]:
data.loc[:,["Class"]] = pd.get_dummies(data.Class,drop_first=True).values

In [20]:
data.describe()

,Comments,Surrounding Code Context,Class
count,11452,11452,11452
unique,7107,7791,2
top,/*********************************************...,-10. // the X-windows Version 11 distribution...,1
freq,144,72,7063


In [21]:
data.isna().sum()

Comments                    0
Surrounding Code Context    0
Class                       0
dtype: int64

In [22]:
# combine code and comment columns to form 1 input column
data["Input"] = data["Comments"] + " " + data["Surrounding Code Context"]
data.head()

,Comments,Surrounding Code Context,Class,Input
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,0,/*test 529*/ -10. int res = 0;\n-9. CURL *...
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",0,"/*test 525*/ -2. fprintf(stderr, ""Usage: l..."
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",0,"/*done*/ -10. multi_add_handle(m, curl);\n-9..."
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,0,/*test 529*/ -10. int res = 0;\n-9. CURL *...
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",0,"/*test 525*/ -2. fprintf(stderr, ""Usage: l..."


In [23]:
from collections import Counter
Y = data['Class']
X = data.drop(columns="Class")
counter = Counter(Y)
print(counter)

Counter({1: 7063, 0: 4389})


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=944fff1e3f52f4cf7b88c7e0fd39f70d0f43a475a36aa30d6f94ab7661d57123
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [24]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("flax-sentence-embeddings/st-codesearch-distilroberta-base")

# Encode our code into the vector space
code_emb = model.encode(data["Input"].astype(str).values.tolist(), convert_to_tensor=True)

In [ ]:
print(code_emb[0])

tensor([ 0.0068, -0.0611,  0.0388,  0.0529,  0.0217, -0.0211, -0.0420,  0.0630,
         0.0126, -0.0125,  0.0713,  0.0211,  0.0106,  0.0454,  0.0125, -0.0556,
         0.0718, -0.0727,  0.0184,  0.0609,  0.0255, -0.0450,  0.0129, -0.0275,
         0.0509,  0.0214, -0.0213,  0.0456, -0.0039, -0.0499,  0.0315,  0.0395,
        -0.0109, -0.0052,  0.0253, -0.0388, -0.0248,  0.0252, -0.0394,  0.0126,
        -0.0330,  0.0567,  0.0336,  0.0121,  0.0094, -0.0014,  0.0196, -0.0147,
        -0.0026, -0.0008,  0.0226, -0.0011, -0.0308,  0.0553, -0.0262,  0.0224,
         0.0421, -0.0478, -0.0359,  0.0458, -0.0497, -0.0304, -0.0025,  0.0024,
         0.0414, -0.0536,  0.0504,  0.0453, -0.0059, -0.0405, -0.0075, -0.0024,
        -0.0444,  0.0100,  0.0372,  0.0012, -0.0415, -0.1171, -0.0213,  0.0184,
         0.0198,  0.0017,  0.0165, -0.0262, -0.0225, -0.0534,  0.0488, -0.0709,
         0.0372,  0.0074,  0.0131, -0.0073, -0.0103,  0.0013,  0.0061,  0.0053,
        -0.0251, -0.0121, -0.0937,  0.00

In [ ]:
# use SMOTE to balance dataset
# and generate enough "Not Useful" comment data
!pip install imbalanced-learn

In [25]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
Y=Y.astype('int')
X = code_emb.tolist()
counter = Counter(Y)
print(counter)
X, y = oversample.fit_resample(X, Y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({1: 7063, 0: 4389})
Counter({0: 7063, 1: 7063})


In [ ]:
# random forest evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = RandomForestClassifier(random_state = 42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([57.25440478, 57.83422947, 57.3429172 , 57.5197587 , 60.14270592,
       59.64366508, 73.29595399, 69.4622848 , 67.49067807, 68.77097464,
       56.72610235, 57.67711902, 56.80229664, 55.86572289, 56.04528403,
       56.36754131, 61.28350925, 63.0823276 , 55.86354589, 54.04394245,
       56.03915834, 54.18628097, 55.68704748, 54.4905951 , 56.13476229,
       55.35322428, 55.8082633 , 55.15687132, 55.64643025, 53.93826842]), 'score_time': array([0.14978552, 0.16606188, 0.15117621, 0.15002346, 0.14739609,
       0.15525532, 0.40032339, 0.46599817, 0.15517521, 0.15559268,
       0.44554853, 0.14972138, 0.15763426, 0.1524179 , 0.15069914,
       0.1708312 , 0.14999628, 0.29793763, 0.16567922, 0.32556939,
       0.1721487 , 0.22288656, 0.15420508, 0.14679289, 0.15916109,
       0.15191102, 0.15554929, 0.15273952, 0.14856339, 0.09570003]), 'test_precision': array([0.83561644, 0.84531886, 0.83469945, 0.8340081 , 0.8454039 ,
       0.84027778, 0.84173669, 0.82376502,

In [26]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# define model
model = RandomForestClassifier(random_state = 42)

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

*** K-Fold Split Results ***
{'0': {'precision': 0.8408759124087591, 'recall': 0.8147100424328148, 'f1-score': 0.8275862068965517, 'support': 707}, '1': {'precision': 0.820054945054945, 'recall': 0.8456090651558074, 'f1-score': 0.8326359832635983, 'support': 706}, 'accuracy': 0.8301486199575372, 'macro avg': {'precision': 0.830465428731852, 'recall': 0.8301595537943111, 'f1-score': 0.830111095080075, 'support': 1413}, 'weighted avg': {'precision': 0.8304727963777665, 'recall': 0.8301486199575372, 'f1-score': 0.8301093081811484, 'support': 1413}}
*** K-Fold Split Results ***
{'0': {'precision': 0.8723723723723724, 'recall': 0.8217821782178217, 'f1-score': 0.8463219227967954, 'support': 707}, '1': {'precision': 0.8313253012048193, 'recall': 0.8796033994334278, 'f1-score': 0.854783207157605, 'support': 706}, 'accuracy': 0.8506723283793347, 'macro avg': {'precision': 0.8518488367885959, 'recall': 0.8506927888256248, 'f1-score': 0.8505525649772002, 'support': 1413}, 'weighted avg': {'precis

In [27]:
# calculate average metrics for submission
import numpy
final_dict = {}
final_dict['0-precision'] = []
final_dict['0-recall'] = []
final_dict['0-f1'] = []
final_dict['1-precision'] = []
final_dict['1-recall'] = []
final_dict['1-f1'] = []
final_dict['accuracy'] = []

for dictionary in classification_report_list:
    for key, inner_dict in dictionary.items():
        if key == '0':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['0-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['0-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['0-f1'].append(inner_dict['f1-score'])
        if key == '1':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['1-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['1-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['1-f1'].append(inner_dict['f1-score'])
        if key == "accuracy":
            final_dict['accuracy'].append(dictionary['accuracy'])


for key in final_dict.keys():
    final_dict[key] = numpy.average(final_dict[key])

print("******** FINAL DICT ********")
print(final_dict)

******** FINAL DICT ********
{'0-precision': 0.857580812422668, '0-recall': 0.8265153670365014, '0-f1': 0.8416852017962309, '1-precision': 0.8326387544851702, '1-recall': 0.86257384338191, '1-f1': 0.8472738482153134, 'accuracy': 0.8445420475057255}


In [ ]:
# logistic evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = LogisticRegression(random_state = 42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([3.7595315 , 3.7526269 , 2.18612647, 2.29225564, 2.62626052,
       2.26049089, 4.35855651, 3.66726184, 2.17899609, 2.54246712,
       2.32606626, 2.40179706, 4.22317719, 3.18432331, 2.49445772,
       2.47898436, 2.58372641, 2.43346143, 4.36194396, 3.25201941,
       2.49387264, 2.29506588, 2.91750741, 2.985147  , 4.33724451,
       3.6194129 , 3.78106213, 4.18181276, 4.63927603, 3.34985781]), 'score_time': array([0.20349431, 0.14024615, 0.15434456, 0.14341402, 0.13696957,
       0.205127  , 0.22757125, 0.13729477, 0.13356137, 0.14744377,
       0.15140796, 0.15050364, 0.23771358, 0.13884211, 0.14015341,
       0.1412766 , 0.13741827, 0.28267932, 0.13950968, 0.24811149,
       0.14285541, 0.25418758, 0.15813804, 0.46051908, 0.14117908,
       0.1738205 , 0.16378331, 0.47143745, 0.18451762, 0.17149949]), 'test_precision': array([0.80142857, 0.80796586, 0.81313869, 0.83808096, 0.82404692,
       0.81647399, 0.82985075, 0.81118881, 0.80323054, 0.79387187,
     

In [ ]:
# SVC evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = SVC(random_state = 42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

KeyboardInterrupt: ignored

In [ ]:
# Linear SVC evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = LinearSVC(random_state = 42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([3.68170357, 2.98074913, 3.26308155, 3.04008532, 4.82865644,
       4.92765164, 3.24638677, 2.90235925, 3.02387142, 3.37687683,
       4.7091682 , 3.49832797, 3.16260552, 3.19177651, 4.00162101,
       5.04437923, 3.28160286, 3.1807971 , 3.24634051, 2.95530057,
       4.95452952, 3.88807559, 3.17767072, 2.92164993, 3.99757648,
       4.27970362, 3.10581827, 3.09029055, 2.94304895, 2.79681516]), 'score_time': array([0.14690185, 0.21880245, 0.14619517, 0.20266533, 0.27179909,
       0.22814965, 0.14678621, 0.17042565, 0.14832258, 0.39097261,
       0.16014433, 0.23251796, 0.14384866, 0.23276186, 0.30009484,
       0.21610045, 0.14729166, 0.14055014, 0.14985847, 0.24514246,
       0.14717317, 0.15538168, 0.13972712, 0.13350177, 0.29192948,
       0.33177233, 0.14573956, 0.28861475, 0.14467192, 0.07843447]), 'test_precision': array([0.84428571, 0.83898305, 0.85131195, 0.87408492, 0.85632184,
       0.85507246, 0.8637037 , 0.83264177, 0.84240688, 0.82632293,
     

In [ ]:
# knn evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = KNeighborsClassifier()
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([1.90377927, 1.69835687, 2.21995378, 2.54127884, 1.43840003,
       1.27885723, 1.58075571, 1.25373244, 2.39076686, 1.57651138,
       1.4587183 , 1.27614236, 1.47626019, 1.62739706, 1.4425962 ,
       1.35010147, 1.53193069, 1.29402971, 2.90488243, 2.28199911,
       1.45495057, 1.29860806, 1.51204419, 1.27572131, 1.94700575,
       1.31153154, 2.43555093, 2.51347232, 2.30270052, 1.55818701]), 'score_time': array([2.19197989, 2.23005962, 3.09159493, 2.54169703, 1.7143805 ,
       1.9211812 , 1.76042557, 2.57338357, 1.72916102, 1.77308607,
       1.75972342, 1.78268862, 3.12050223, 3.27741957, 1.79502654,
       1.75151968, 1.68102837, 1.69592357, 2.08898902, 1.94950938,
       1.733603  , 1.91593194, 1.98640156, 3.36626458, 1.71963501,
       2.03476787, 2.95035601, 2.66509461, 1.6667974 , 1.44937921]), 'test_precision': array([0.858473  , 0.85818182, 0.89607843, 0.86751361, 0.91056911,
       0.87549407, 0.88122605, 0.88846881, 0.9022869 , 0.87429644,
     

In [ ]:
# adaboost evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = AdaBoostClassifier(random_state = 42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([105.54068613, 105.26689839, 105.28089356, 105.40180492,
       107.84946942, 107.71821141, 102.57548738, 102.79224849,
       106.83076787, 107.35504198, 104.58415866, 105.6507802 ,
       103.20806146, 105.20248342, 106.34847355, 103.18119478,
       105.83648562, 106.2560823 , 107.30567312, 105.7116766 ,
       103.38895845, 103.26471281, 106.01803613, 107.75825095,
       105.27446461, 105.69495106, 104.4839046 , 106.68334341,
       107.99886918, 105.17520499]), 'score_time': array([0.27323294, 0.23670197, 0.24615431, 0.43016434, 0.24394727,
       0.23455215, 0.25787735, 0.22631526, 0.23271585, 0.24132729,
       0.24486184, 0.25572777, 0.24344397, 0.54232764, 0.2398169 ,
       0.24237156, 0.23909616, 0.49709153, 0.24343395, 0.27040052,
       0.24137092, 0.25354671, 0.4819603 , 0.23724103, 0.25838161,
       0.24816608, 0.24177098, 0.23844528, 0.24246931, 0.13913107]), 'test_precision': array([0.75147929, 0.76453488, 0.74053296, 0.77548006, 0.76126126

In [ ]:
# passiveaggressiveclassifier evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = PassiveAggressiveClassifier(random_state = 42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([2.05913067, 1.79651356, 2.23692966, 2.42080569, 2.77512312,
       2.3045547 , 2.03726268, 2.74677372, 2.61077023, 2.10115886,
       3.38741684, 2.66523027, 2.48915768, 2.51042366, 2.07530499,
       3.26309395, 3.52758765, 2.69896102, 1.73158145, 2.11934471,
       2.23065782, 2.86521673, 2.3513391 , 2.88002491, 1.96823955,
       2.66263008, 2.48644543, 4.75810289, 3.45556307, 1.71263027]), 'score_time': array([0.13396573, 0.15426564, 0.15399528, 0.39406157, 0.12667036,
       0.14266992, 0.13220263, 0.15387702, 0.13262224, 0.45516539,
       0.13976431, 0.13375735, 0.12969947, 0.14765835, 0.16593647,
       0.30101156, 0.25630355, 0.13130713, 0.27455211, 0.12893558,
       0.2300086 , 0.30243778, 0.31342554, 0.13252163, 0.13108206,
       0.31021261, 0.30378819, 0.27954435, 0.23528433, 0.07753778]), 'test_precision': array([0.89648799, 0.88764045, 0.80713342, 0.91369606, 0.87106918,
       0.74944072, 0.8175853 , 0.85692542, 0.95987654, 0.71022129,
     

In [ ]:
# voting classifier evaluated on re-balanced dataset

from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = ['precision', 'recall', 'f1']

# prepare candidate classifiers
rf_model = RandomForestClassifier(random_state = 42)
lr_model = LogisticRegression(random_state = 42)
svc_model = LinearSVC(random_state = 42)
pac_model = PassiveAggressiveClassifier(random_state = 42)

# define model
model =  VotingClassifier(estimators=[('rf', rf_model), ('lr', lr_model), ('svc', svc_model), ('pac', pac_model)], voting='hard')
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([63.70453596, 63.70869827, 65.23361254, 65.20233679, 65.29234147,
       65.78260112, 62.26571584, 64.12142968, 66.20328879, 64.8923583 ,
       64.10616159, 63.49430943, 64.45932937, 64.25569963, 62.1320343 ,
       65.06257319, 68.02834988, 72.78442812, 64.29279375, 62.47790003,
       62.01369572, 64.57836485, 64.28973007, 64.65200686, 62.36717892,
       64.76543045, 64.52865148, 64.56748366, 60.67115164, 60.2023778 ]), 'score_time': array([0.57360125, 0.86824274, 0.55736923, 0.84713078, 0.61063862,
       0.58011007, 0.62876177, 1.08489347, 0.63977623, 0.59210658,
       1.11570072, 1.17290044, 0.5574615 , 0.92915058, 1.30681229,
       0.63448429, 1.2960825 , 0.87908816, 0.62956095, 0.60090709,
       1.04053402, 0.59243059, 0.64625096, 0.59601712, 0.98766637,
       0.59822941, 0.63554668, 0.67316747, 1.23250937, 0.30660796]), 'test_precision': array([0.89918699, 0.87405159, 0.87347561, 0.90609555, 0.87538462,
       0.86297376, 0.88601824, 0.85882353,

In [ ]:
# neural network evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([37.03542924, 36.34947205, 41.10257602, 41.43582749, 35.80289102,
       37.45578885, 36.35295081, 35.79368353, 37.57451892, 37.60090017,
       41.27088189, 43.05542207, 36.14631987, 35.57429242, 36.96601129,
       37.31169534, 35.59765935, 35.61085033, 39.84781003, 38.41193986,
       36.75185132, 31.15179014, 37.34583354, 35.28114963, 36.51735759,
       37.68074512, 35.76828122, 35.42355347, 39.06646919, 39.03307867]), 'score_time': array([0.15719104, 0.17889023, 0.15678287, 0.14106226, 0.31544709,
       0.29294157, 0.15537834, 0.14274669, 0.1642189 , 0.14178038,
       0.33967948, 0.27077961, 0.17163825, 0.12361741, 0.15373945,
       0.15031886, 0.15936923, 0.22706175, 0.1588335 , 0.15083432,
       0.2854991 , 0.1556344 , 0.18381977, 0.15876627, 0.3356111 ,
       0.15835166, 0.17633963, 0.15282369, 0.20818758, 0.07890368]), 'test_precision': array([0.89371257, 0.90072993, 0.90117994, 0.8751773 , 0.89064748,
       0.91004498, 0.88209607, 0.88809182,

In [28]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# define model
model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8596491228070176, 'recall': 0.900990099009901, 'f1-score': 0.8798342541436465, 'support': 707}, '1': {'precision': 0.8958333333333334, 'recall': 0.8526912181303116, 'f1-score': 0.8737300435413642, 'support': 706}, 'accuracy': 0.8768577494692145, 'macro avg': {'precision': 0.8777412280701755, 'recall': 0.8768406585701063, 'f1-score': 0.8767821488425054, 'support': 1413}, 'weighted avg': {'precision': 0.8777284240324804, 'recall': 0.8768577494692145, 'f1-score': 0.8767843088604114, 'support': 1413}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.891213389121339, 'recall': 0.9038189533239038, 'f1-score': 0.8974719101123596, 'support': 707}, '1': {'precision': 0.9022988505747126, 'recall': 0.8895184135977338, 'f1-score': 0.8958630527817403, 'support': 706}, 'accuracy': 0.8966737438075018, 'macro avg': {'precision': 0.8967561198480258, 'recall': 0.8966686834608188, 'f1-score': 0.8966674814470499, 'support': 1413}, 'weighted avg': {'precision': 0.8967521971794294, 'recall': 0.8966737438075018, 'f1-score': 0.8966680507525455, 'support': 1413}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8527027027027027, 'recall': 0.8937677053824362, 'f1-score': 0.8727524204702627, 'support': 706}, '1': {'precision': 0.8883928571428571, 'recall': 0.8456090651558074, 'f1-score': 0.8664731494920174, 'support': 706}, 'accuracy': 0.8696883852691218, 'macro avg': {'precision': 0.8705477799227799, 'recall': 0.8696883852691217, 'f1-score': 0.86961278498114, 'support': 1412}, 'weighted avg': {'precision': 0.87054777992278, 'recall': 0.8696883852691218, 'f1-score': 0.86961278498114, 'support': 1412}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8821022727272727, 'recall': 0.8796033994334278, 'f1-score': 0.8808510638297872, 'support': 706}, '1': {'precision': 0.8799435028248588, 'recall': 0.8824362606232294, 'f1-score': 0.8811881188118812, 'support': 706}, 'accuracy': 0.8810198300283286, 'macro avg': {'precision': 0.8810228877760657, 'recall': 0.8810198300283286, 'f1-score': 0.8810195913208342, 'support': 1412}, 'weighted avg': {'precision': 0.8810228877760657, 'recall': 0.8810198300283286, 'f1-score': 0.8810195913208342, 'support': 1412}}
*** K-Fold Split Results ***
{'0': {'precision': 0.9035087719298246, 'recall': 0.8753541076487252, 'f1-score': 0.8892086330935253, 'support': 706}, '1': {'precision': 0.8791208791208791, 'recall': 0.9065155807365439, 'f1-score': 0.8926080892608089, 'support': 706}, 'accuracy': 0.8909348441926346, 'macro avg': {'precision': 0.8913148255253518, 'recall': 0.8909348441926346, 'f1-score': 0.8909083611771671, 'support': 1412}, 'weighted avg': {'pr

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [29]:
# calculate average metrics for submission
import numpy
final_dict = {}
final_dict['0-precision'] = []
final_dict['0-recall'] = []
final_dict['0-f1'] = []
final_dict['1-precision'] = []
final_dict['1-recall'] = []
final_dict['1-f1'] = []
final_dict['accuracy'] = []

for dictionary in classification_report_list:
    for key, inner_dict in dictionary.items():
        if key == '0':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['0-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['0-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['0-f1'].append(inner_dict['f1-score'])
        if key == '1':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['1-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['1-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['1-f1'].append(inner_dict['f1-score'])
        if key == "accuracy":
            final_dict['accuracy'].append(dictionary['accuracy'])


for key in final_dict.keys():
    final_dict[key] = numpy.average(final_dict[key])

print("******** FINAL DICT ********")
print(final_dict)

******** FINAL DICT ********
{'0-precision': 0.8769283716834008, '0-recall': 0.896930132106695, '0-f1': 0.886644414903265, '1-precision': 0.8948466409552037, '1-recall': 0.8738064518714114, '1-f1': 0.8840103780696502, 'accuracy': 0.8853650040397846}


In [ ]:
# save model
import pickle
pkl_filename = 'voting_classifier_with_chatgpt_data.pkl'
with open(pkl_filename, 'wb') as file:
  pickle.dump(model, file)

# load model
"""
with open(pkl_filename, 'rb') as file:
	pickle_model = pickle.load(file)
"""

"\nwith open(pkl_filename, 'rb') as file:\n\tpickle_model = pickle.load(file)\n"

In [ ]:
# voting classifier evaluated on re-balanced dataset

from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = ['precision', 'recall', 'f1']

# prepare candidate classifiers
rf_model = RandomForestClassifier(random_state = 42)
nn_model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)
svc_model = LinearSVC(random_state = 42)

# define model
model =  VotingClassifier(estimators=[('rf', rf_model), ('nn', nn_model), ('svc', svc_model)], voting='hard')
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([95.37238693, 95.50000882, 97.50235391, 96.26758766, 97.5069046 ,
       96.19780254, 93.60567284, 97.41898942, 94.76310158, 93.97464061,
       96.18827391, 96.91014314, 93.54418635, 98.11220121, 91.29829431,
       96.99753666, 96.32576132, 97.28917718, 94.84302592, 94.31824183,
       93.49002314, 95.34722614, 95.93378043, 97.63210559, 94.01193237,
       96.9354105 , 92.94612122, 94.87039518, 95.49468207, 87.69752765]), 'score_time': array([0.47466207, 0.58083487, 0.4802072 , 0.56627655, 0.47927761,
       0.46369314, 0.86930561, 0.49369335, 0.42710376, 0.48831272,
       0.5130496 , 0.47988319, 0.92850089, 0.49995518, 0.45764518,
       0.49163747, 0.47573304, 0.47123551, 0.94216728, 0.4735868 ,
       0.50030947, 0.47378182, 0.48729491, 0.47416019, 0.94242287,
       0.47420144, 0.47327065, 0.91763139, 0.47723413, 0.44091487]), 'test_precision': array([0.86901408, 0.89316239, 0.89714286, 0.89511494, 0.87535014,
       0.89705882, 0.89048991, 0.87136929,

In [30]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# prepare candidate classifiers
rf_model = RandomForestClassifier(random_state = 42)
nn_model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)
svc_model = LinearSVC(random_state = 42)

# define model
model =  VotingClassifier(estimators=[('rf', rf_model), ('nn', nn_model), ('svc', svc_model)], voting='hard')

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8685314685314686, 'recall': 0.8783592644978784, 'f1-score': 0.8734177215189873, 'support': 707}, '1': {'precision': 0.8767908309455588, 'recall': 0.8668555240793201, 'f1-score': 0.8717948717948717, 'support': 706}, 'accuracy': 0.8726114649681529, 'macro avg': {'precision': 0.8726611497385137, 'recall': 0.8726073942885992, 'f1-score': 0.8726062966569295, 'support': 1413}, 'weighted avg': {'precision': 0.872658227104963, 'recall': 0.8726114649681529, 'f1-score': 0.8726068709137321, 'support': 1413}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8982558139534884, 'recall': 0.8741159830268741, 'f1-score': 0.8860215053763442, 'support': 707}, '1': {'precision': 0.8772413793103448, 'recall': 0.9008498583569405, 'f1-score': 0.888888888888889, 'support': 706}, 'accuracy': 0.8874734607218684, 'macro avg': {'precision': 0.8877485966319166, 'recall': 0.8874829206919073, 'f1-score': 0.8874551971326166, 'support': 1413}, 'weighted avg': {'precision': 0.8877560327375936, 'recall': 0.8874734607218684, 'f1-score': 0.8874541824887693, 'support': 1413}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

*** K-Fold Split Results ***
{'0': {'precision': 0.8649025069637883, 'recall': 0.8796033994334278, 'f1-score': 0.872191011235955, 'support': 706}, '1': {'precision': 0.877521613832853, 'recall': 0.8626062322946175, 'f1-score': 0.87, 'support': 706}, 'accuracy': 0.8711048158640227, 'macro avg': {'precision': 0.8712120603983207, 'recall': 0.8711048158640227, 'f1-score': 0.8710955056179774, 'support': 1412}, 'weighted avg': {'precision': 0.8712120603983207, 'recall': 0.8711048158640227, 'f1-score': 0.8710955056179777, 'support': 1412}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8832853025936599, 'recall': 0.8682719546742209, 'f1-score': 0.8757142857142857, 'support': 706}, '1': {'precision': 0.8704735376044568, 'recall': 0.8852691218130312, 'f1-score': 0.877808988764045, 'support': 706}, 'accuracy': 0.876770538243626, 'macro avg': {'precision': 0.8768794200990584, 'recall': 0.876770538243626, 'f1-score': 0.8767616372391653, 'support': 1412}, 'weighted avg': {'precision': 0.8768794200990584, 'recall': 0.876770538243626, 'f1-score': 0.8767616372391653, 'support': 1412}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8945827232796486, 'recall': 0.8654390934844193, 'f1-score': 0.8797696184305256, 'support': 706}, '1': {'precision': 0.869684499314129, 'recall': 0.8980169971671388, 'f1-score': 0.883623693379791, 'support': 706}, 'accuracy': 0.8817280453257791, 'macro avg': {'precision': 0.8821336112968887, 'recall': 0.881728045325779, 'f1-score': 0.8816966559051582, 'support': 1412}, 'weighted avg': {'precision': 0.8821336112968888, 'recall': 0.8817280453257791, 'f1-score': 0.8816966559051582, 'support': 1412}}


In [31]:
# calculate average metrics for submission
import numpy
final_dict = {}
final_dict['0-precision'] = []
final_dict['0-recall'] = []
final_dict['0-f1'] = []
final_dict['1-precision'] = []
final_dict['1-recall'] = []
final_dict['1-f1'] = []
final_dict['accuracy'] = []

for dictionary in classification_report_list:
    for key, inner_dict in dictionary.items():
        if key == '0':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['0-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['0-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['0-f1'].append(inner_dict['f1-score'])
        if key == '1':
            for inner_key in inner_dict.keys():
                if inner_key == "precision":
                    final_dict['1-precision'].append(inner_dict['precision'])
                if inner_key == "recall":
                    final_dict['1-recall'].append(inner_dict['recall'])
                if inner_key == "f1-score":
                    final_dict['1-f1'].append(inner_dict['f1-score'])
        if key == "accuracy":
            final_dict['accuracy'].append(dictionary['accuracy'])


for key in final_dict.keys():
    final_dict[key] = numpy.average(final_dict[key])

print("******** FINAL DICT ********")
print(final_dict)

******** FINAL DICT ********
{'0-precision': 0.882111701610263, '0-recall': 0.8803164229818369, '0-f1': 0.8811108449706349, '1-precision': 0.8807138176181707, '1-recall': 0.8821587176928943, '1-f1': 0.8813338209007479, 'accuracy': 0.8812356527509628}


In [6]:
data = pd.read_csv('FIRE2023_IRSE_training_Code_Comment_Seed_Data.csv')
data.head(5)

,Comments,Surrounding Code Context,Class
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful


In [7]:
# experiment 2: Try self-training with ChatGPT generated data
chatgpt_data = pd.read_csv('/content/chatgpt_generated_data.csv')
chatgpt_data.head(5)

,Line of Code,Comment,Class
0,int randomNumber = rand() % 100 + 1;,"""Generate a random number between 1 and 100.""",Useful
1,int length = strlen(inputString);,"""Get the length of the input string.""",Useful
2,for (int i = 0; i < 10; i++) {,"""Loop from 0 to 9.""",Useful
3,"printf(%d "", i);""","""Print the current value of i.""",Useful
4,"char name[20] = ""John Smith"";","""Declare and initialize a character array to ...",Useful


In [8]:
# clean dataset by removing extra quotation marks that ChatGPT adds when it generates content
chatgpt_data["Line of Code"] = chatgpt_data["Line of Code"].str.lstrip('"')
chatgpt_data["Line of Code"] = chatgpt_data["Line of Code"].str.rstrip('"')
# rename columns to match original data columns
chatgpt_data = chatgpt_data.rename(columns={'Comment': 'Comments', 'Line of Code': 'Surrounding Code Context', 'Class': 'Class'})
# change column order to match original dataset
chatgpt_data = chatgpt_data[['Comments', 'Surrounding Code Context', 'Class']]
chatgpt_data['Comments'] = chatgpt_data['Comments'].map(lambda x: x.strip().lstrip('"').rstrip('"'))
chatgpt_data['Surrounding Code Context'] = chatgpt_data['Surrounding Code Context'].map(lambda x: x.strip())
chatgpt_data['Class'] = chatgpt_data['Class'].map(lambda x: x.strip())
chatgpt_data.head(5)

,Comments,Surrounding Code Context,Class
0,Generate a random number between 1 and 100.,int randomNumber = rand() % 100 + 1;,Useful
1,Get the length of the input string.,int length = strlen(inputString);,Useful
2,Loop from 0 to 9.,for (int i = 0; i < 10; i++) {,Useful
3,Print the current value of i.,"printf(%d "", i);",Useful
4,Declare and initialize a character array to st...,"char name[20] = ""John Smith"";",Useful


In [9]:
# now merge new dataset with the old
full_data = data.append(chatgpt_data)
print(full_data.shape)
full_data.head(5)

(11872, 3)


<ipython-input-9-a160ff6290b5>:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  full_data = data.append(chatgpt_data)


,Comments,Surrounding Code Context,Class
0,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
1,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful
2,/*done*/,"-10. multi_add_handle(m, curl);\n-9. for(;...",Not Useful
3,/*test 529*/,-10. int res = 0;\n-9. CURL *curl = NULL;\...,Not Useful
4,/*test 525*/,"-2. fprintf(stderr, ""Usage: lib529 [url] [...",Not Useful


In [10]:
# now re-apply data transformation on entire dataset and prepare for training
full_data.loc[:,["Class"]] = pd.get_dummies(full_data.Class,drop_first=True).values

full_data["Input"] = full_data["Comments"] + " " + full_data["Surrounding Code Context"]

<ipython-input-10-f5789c578e43>:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  full_data.loc[:,["Class"]] = pd.get_dummies(full_data.Class,drop_first=True).values


In [11]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 79.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 119.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.6 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=b2332fa2142c554a5d1192276a36999369e228263f634d109e3d6acc5141d993
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [12]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("flax-sentence-embeddings/st-codesearch-distilroberta-base")

# Encode our code into the vector space
code_emb = model.encode(full_data["Input"].astype(str).values.tolist(), convert_to_tensor=True)

In [13]:
# use SMOTE to balance dataset
# and generate enough "Not Useful" comment data
!pip install imbalanced-learn

In [14]:
from collections import Counter
from imblearn.over_sampling import SMOTE
oversample = SMOTE()
Y = full_data['Class']
Y=Y.astype('int')
X = code_emb.tolist()
counter = Counter(Y)
print(counter)
X, y = oversample.fit_resample(X, Y)
# summarize the new class distribution
counter = Counter(y)
print(counter)

Counter({1: 7474, 0: 4398})
Counter({0: 7474, 1: 7474})


In [ ]:
# evaluate self-training semi-supervised classifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# define scoring metrics
scoring = ['precision', 'recall', 'f1']

# prepare candidate classifiers
rf_model = RandomForestClassifier(random_state = 42)
nn_model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)
svc_model = LinearSVC(random_state = 42)

# define model
model =  VotingClassifier(estimators=[('rf', rf_model), ('nn', nn_model), ('svc', svc_model)], voting='hard')
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([99.46883821, 99.51684141, 99.5052731 , 98.60438585, 98.41646934,
       97.5736649 , 97.89818382, 90.47905087, 96.11927915, 98.26053476,
       92.04900098, 96.87496996, 97.44736481, 95.9308064 , 96.14342713,
       97.1648159 , 95.61653733, 97.25824046, 97.35290837, 97.55585647,
       97.11324239, 97.9244473 , 98.03323007, 97.49427748, 98.34804225,
       95.1361742 , 97.29412103, 96.6040659 , 96.56746435, 93.81726193]), 'score_time': array([0.44869399, 0.60086679, 0.44434237, 0.45239806, 0.45886683,
       0.46888185, 0.4571569 , 0.47311926, 0.47701955, 0.46648669,
       0.47689533, 0.70804715, 0.77470469, 0.68178606, 0.94308352,
       0.44502759, 0.47321129, 0.44069719, 0.51102781, 0.44551921,
       0.49328041, 0.44459343, 0.46812367, 0.44668841, 0.46539521,
       0.91634154, 0.87873936, 0.45649147, 0.90287638, 0.24239707]), 'test_precision': array([0.87919463, 0.88681758, 0.87765957, 0.89617486, 0.91083676,
       0.8804205 , 0.88582677, 0.89697802,

In [31]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# prepare candidate classifiers
rf_model = RandomForestClassifier(random_state = 42)
nn_model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)
svc_model = LinearSVC(random_state = 42)

# define model
model =  VotingClassifier(estimators=[('rf', rf_model), ('nn', nn_model), ('svc', svc_model)], voting='hard')

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8810160427807486, 'recall': 0.8810160427807486, 'f1-score': 0.8810160427807486, 'support': 748}, '1': {'precision': 0.8808567603748326, 'recall': 0.8808567603748326, 'f1-score': 0.8808567603748326, 'support': 747}, 'accuracy': 0.8809364548494983, 'macro avg': {'precision': 0.8809364015777906, 'recall': 0.8809364015777906, 'f1-score': 0.8809364015777906, 'support': 1495}, 'weighted avg': {'precision': 0.8809364548494983, 'recall': 0.8809364548494983, 'f1-score': 0.8809364548494983, 'support': 1495}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.9016172506738545, 'recall': 0.8943850267379679, 'f1-score': 0.897986577181208, 'support': 748}, '1': {'precision': 0.8950863213811421, 'recall': 0.9022757697456493, 'f1-score': 0.8986666666666667, 'support': 747}, 'accuracy': 0.8983277591973244, 'macro avg': {'precision': 0.8983517860274983, 'recall': 0.8983303982418086, 'f1-score': 0.8983266219239374, 'support': 1495}, 'weighted avg': {'precision': 0.8983539702847867, 'recall': 0.8983277591973244, 'f1-score': 0.89832639446926, 'support': 1495}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

*** K-Fold Split Results ***
{'0': {'precision': 0.8955431754874652, 'recall': 0.8607764390896921, 'f1-score': 0.8778156996587031, 'support': 747}, '1': {'precision': 0.8661518661518661, 'recall': 0.8997326203208557, 'f1-score': 0.882622950819672, 'support': 748}, 'accuracy': 0.8802675585284281, 'macro avg': {'precision': 0.8808475208196657, 'recall': 0.880254529705274, 'f1-score': 0.8802193252391876, 'support': 1495}, 'weighted avg': {'precision': 0.8808376909503226, 'recall': 0.8802675585284281, 'f1-score': 0.8802209330154955, 'support': 1495}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8727034120734908, 'recall': 0.8902275769745649, 'f1-score': 0.881378396288933, 'support': 747}, '1': {'precision': 0.888130968622101, 'recall': 0.8703208556149733, 'f1-score': 0.8791357191087104, 'support': 748}, 'accuracy': 0.8802675585284281, 'macro avg': {'precision': 0.8804171903477959, 'recall': 0.8802742162947691, 'f1-score': 0.8802570576988217, 'support': 1495}, 'weighted avg': {'precision': 0.8804223500657051, 'recall': 0.8802675585284281, 'f1-score': 0.8802563076395641, 'support': 1495}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.889958734525447, 'recall': 0.8661311914323963, 'f1-score': 0.8778833107191316, 'support': 747}, '1': {'precision': 0.8696219035202086, 'recall': 0.892904953145917, 'f1-score': 0.8811096433289299, 'support': 747}, 'accuracy': 0.8795180722891566, 'macro avg': {'precision': 0.8797903190228278, 'recall': 0.8795180722891567, 'f1-score': 0.8794964770240308, 'support': 1494}, 'weighted avg': {'precision': 0.8797903190228278, 'recall': 0.8795180722891566, 'f1-score': 0.8794964770240308, 'support': 1494}}
*** K-Fold Split Results ***
{'0': {'precision': 0.8771466314398944, 'recall': 0.8888888888888888, 'f1-score': 0.8829787234042553, 'support': 747}, '1': {'precision': 0.8873812754409769, 'recall': 0.8755020080321285, 'f1-score': 0.8814016172506739, 'support': 747}, 'accuracy': 0.8821954484605087, 'macro avg': {'precision': 0.8822639534404356, 'recall': 0.8821954484605087, 'f1-score': 0.8821901703274646, 'support': 1494}, 'weighted avg': {'prec

In [2]:
# neural network evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

NameError: ignored

In [15]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# define model
model = MLPClassifier(hidden_layer_sizes=(20,10),random_state=42)

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8605263157894737, 'recall': 0.8743315508021391, 'f1-score': 0.8673740053050398, 'support': 748}, '1': {'precision': 0.8721088435374149, 'recall': 0.85809906291834, 'f1-score': 0.8650472334682862, 'support': 747}, 'accuracy': 0.8662207357859532, 'macro avg': {'precision': 0.8663175796634444, 'recall': 0.8662153068602396, 'f1-score': 0.866210619386663, 'support': 1495}, 'weighted avg': {'precision': 0.8663137059083447, 'recall': 0.8662207357859532, 'f1-score': 0.8662113975712238, 'support': 1495}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8585732165206508, 'recall': 0.9171122994652406, 'f1-score': 0.8868778280542986, 'support': 748}, '1': {'precision': 0.9109195402298851, 'recall': 0.8487282463186078, 'f1-score': 0.8787248787248788, 'support': 747}, 'accuracy': 0.882943143812709, 'macro avg': {'precision': 0.8847463783752679, 'recall': 0.8829202728919242, 'f1-score': 0.8828013533895886, 'support': 1495}, 'weighted avg': {'precision': 0.8847288712435926, 'recall': 0.882943143812709, 'f1-score': 0.8828040801284949, 'support': 1495}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.8718274111675127, 'recall': 0.9196787148594378, 'f1-score': 0.895114006514658, 'support': 747}, '1': {'precision': 0.9151343705799151, 'recall': 0.8649732620320856, 'f1-score': 0.8893470790378007, 'support': 748}, 'accuracy': 0.8923076923076924, 'macro avg': {'precision': 0.893480890873714, 'recall': 0.8923259884457617, 'f1-score': 0.8922305427762294, 'support': 1495}, 'weighted avg': {'precision': 0.8934953748066278, 'recall': 0.8923076923076924, 'f1-score': 0.8922286140379428, 'support': 1495}}


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


*** K-Fold Split Results ***
{'0': {'precision': 0.9044943820224719, 'recall': 0.8621151271753681, 'f1-score': 0.8827964359150102, 'support': 747}, '1': {'precision': 0.8682864450127877, 'recall': 0.9089692101740294, 'f1-score': 0.8881621975147156, 'support': 747}, 'accuracy': 0.8855421686746988, 'macro avg': {'precision': 0.8863904135176298, 'recall': 0.8855421686746987, 'f1-score': 0.8854793167148629, 'support': 1494}, 'weighted avg': {'precision': 0.8863904135176298, 'recall': 0.8855421686746988, 'f1-score': 0.885479316714863, 'support': 1494}}
*** K-Fold Split Results ***
{'0': {'precision': 0.8727034120734908, 'recall': 0.8902275769745649, 'f1-score': 0.881378396288933, 'support': 747}, '1': {'precision': 0.8879781420765027, 'recall': 0.8701472556894244, 'f1-score': 0.8789722785665991, 'support': 747}, 'accuracy': 0.8801874163319946, 'macro avg': {'precision': 0.8803407770749967, 'recall': 0.8801874163319947, 'f1-score': 0.8801753374277661, 'support': 1494}, 'weighted avg': {'prec

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [ ]:
# random forest evaluated on re-balanced dataset
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics import make_scorer

# define scoring metrics
scoring = {'accuracy': make_scorer(accuracy_score),
            'precision': make_scorer(precision_score),
           'recall': make_scorer(recall_score),
           'f1': make_scorer(f1_score)}

scoring = ['precision', 'recall', 'f1']

# define model
model = RandomForestClassifier(random_state = 42)
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
scores = cross_validate(model, X, y, scoring= scoring, cv=cv, n_jobs=-1)
print('Scores:', scores)

Scores: {'fit_time': array([59.39355874, 58.6364913 , 58.01466608, 57.32071447, 57.73810697,
       57.02665353, 59.24876976, 57.70379138, 57.560642  , 57.00001836,
       57.38927221, 56.82229185, 57.22845459, 57.17982721, 57.54526782,
       57.52511811, 56.85505939, 57.11272669, 57.21608591, 57.08910751,
       55.17989779, 57.41469836, 56.02990794, 56.87291074, 56.16347885,
       57.03550529, 56.86514759, 57.47136974, 57.48581266, 56.93324327]), 'score_time': array([0.16521811, 0.17102075, 0.15123606, 0.15870738, 0.15656495,
       0.15671682, 0.30239344, 0.27232099, 0.15584278, 0.49145484,
       0.16600251, 0.18694639, 0.17182064, 0.32569861, 0.1517427 ,
       0.18317437, 0.15178394, 0.26011467, 0.24669695, 0.30730534,
       0.28095293, 0.15952778, 0.31818795, 0.16268229, 0.29642057,
       0.15351939, 0.22768283, 0.15335703, 0.16799593, 0.09529448]), 'test_precision': array([0.84876141, 0.8496732 , 0.82496782, 0.83942559, 0.88059701,
       0.84326425, 0.83228247, 0.85278515,

In [16]:
# classification report on CV with VotingClassifier
import numpy as np
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import VotingClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, make_scorer

# define model
model = RandomForestClassifier(random_state = 42)

classification_report_list = []
# evaluate pipeline
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=42)
for train_index, test_index in cv.split(X, y):
    x_train, x_test = np.array(X)[train_index], np.array(X)[test_index]
    y_train, y_test = y[train_index], y[test_index]
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print("*** K-Fold Split Results ***")
    classification_report_results = classification_report(y_test, y_pred, output_dict=True)
    print(classification_report_results)
    classification_report_list.append(classification_report_results)

*** K-Fold Split Results ***
{'0': {'precision': 0.8691860465116279, 'recall': 0.7994652406417112, 'f1-score': 0.8328690807799444, 'support': 748}, '1': {'precision': 0.8141263940520446, 'recall': 0.8795180722891566, 'f1-score': 0.8455598455598454, 'support': 747}, 'accuracy': 0.8394648829431438, 'macro avg': {'precision': 0.8416562202818363, 'recall': 0.8394916564654339, 'f1-score': 0.8392144631698949, 'support': 1495}, 'weighted avg': {'precision': 0.841674634881321, 'recall': 0.8394648829431438, 'f1-score': 0.8392102187669584, 'support': 1495}}
*** K-Fold Split Results ***
{'0': {'precision': 0.8753462603878116, 'recall': 0.8449197860962567, 'f1-score': 0.8598639455782314, 'support': 748}, '1': {'precision': 0.8499353169469599, 'recall': 0.8795180722891566, 'f1-score': 0.8644736842105263, 'support': 747}, 'accuracy': 0.8622073578595317, 'macro avg': {'precision': 0.8626407886673857, 'recall': 0.8622189291927067, 'f1-score': 0.8621688148943789, 'support': 1495}, 'weighted avg': {'pre